In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

# model_name = "speakleash/Bielik-4.5B-v3.0-Instruct"
model_name = "speakleash/Bielik-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16, device_map='auto')#.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [13]:
prompt = open('prompt.txt').read()

messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "Tekst do poprawy: Krzesło. Rodza jednoosobowego siedziska z oparciem przez rośnie o konstrukcji szkieletowej. Niektóre źródła do definicji krzesła dodają określenie bez podwokietników, uważając ich zastosowanie za charakterystyczną cechę fotela. Inne źródła używają konsekwentnie w formie krzesła z podwokietnikami. Jest jednym z najstarszych sprzętów w branży meblarskiej. W swojej historii przechodziły najwięcej zmian formalnych w XVII i XVIII wieku. Stąd na przykład krzesła w stylu ludzika 14, 15 i 16 różnią się tak znacznie, a są odświeżone tylko w czasie."},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

encoded = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True
)

input_ids = encoded.input_ids#.to(device)
attention_mask = encoded.attention_mask#.to(device)

generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7
)

decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
Zadanie: Otrzymasz tekst wygenerowany przez system rozpoznawania mowy (STT). Tekst może zawierać błędy językowe, fonetyczne, składniowe lub logiczne. Twoim zadaniem jest:
1. Poprawić błędy ortograficzne, gramatyczne, interpunkcyjne i stylistyczne.
2. Zachować oryginalny sens wypowiedzi.
3. Nie dodawać nowych informacji ani nie zmieniać znaczenia.
4. Jeśli coś jest niejasne, popraw to na najbardziej prawdopodobną poprawną formę.
Po poprawkach zwróć tylko poprawiony tekst, bez komentarzy.

Przykład:
Tekst do poprawy: "Krzesło bez podwokietników"
Tekst poprawiony: "Krzesło bez podłokietników"

<</SYS>>

Tekst do poprawy: Krzesło. Rodza jednoosobowego siedziska z oparciem przez rośnie o konstrukcji szkieletowej. Niektóre źródła do definicji krzesła dodają określenie bez podwokietników, uważając ich zastosowanie za charakterystyczną cechę fotela. Inne źródła używają konsekwentnie w formie krzesła z podwokietnikami. Jest jednym z najstarszych sprzętów w branży meblarskiej. W s

In [4]:
import os
print(os.getcwd())

/content
